# Find MAST Observations of Targets in Rocky Worlds DDT "Targets Under Consideration" List
---
**Author**: Mees Fix (mfix@stsci.edu) <br>
**Latest Update**: 31 March 2025<br>
**Data**: None<br>

<a id='intro'></a>
## Introduction


The examples below use the `archive_utils.py` file in the Rocky Worlds DDT Python API to discover observations of targets under consideration that might already exist. In this notebook we will use two targets under consideration: `LTT 1445 A b` and `TRAPPIST-1 g`.

* Import Rocky Worlds DDT utility functions
* Obtain exoplanet metadata from NASA NexSci Exoplanet Archive (ra, dec, period, planetary ephemeris).
* Obtain JWST Observations of the same region (potential transits, phase curves, secondary transits).
* Use JWST observation metadata of observation start and end times along with the targets period and planetary ephemeris to see if there are potential exoplanet events.
* Build an astropy table and add the orbit, phase and potential exoplanet event around the target

<a id='top'></a>
## Table of Contents
* [Imports](#imports)
* [Query NexSci Archive (LTT 1445 A b)](#ltt1445ab)
    * [Reviewing values returned from our API queries.](#ltt1445ab_values)
    * [Query for JWST Observations of Target](#mastquery_ltt1445ab)
    * [LTT 1445 A b Secondary Eclipse Results](#ltt1445ab_results)
* [Query NexSci Archive (Trappist-1 g)](#trappist1g)
    * [Reviewing values returned from our API queries.](#trappist1g_values)
    * [Selecting Valid Values for query_mast_jwst_archive](#valid_values)
    * [LTT 1445 A b Secondary Eclipse Results](#trappist1g_results)
* [All Events for Target Under Consideration](#all_events_for_tuc)

<a id='imports'></a>
## Imports

In [ ]:
from pathlib import Path
from astropy.io import ascii
from astropy.table import vstack
import numpy as np

from rocky_worlds_utils.archive_utils import check_jwst_event_type, query_nexsci_archive, check_jwst_observations

<a id='ltt1445ab'></a>
## Query NexSci Archive (LTT 1445 A b)

In [ ]:
target_name = "LTT 1445 A b"

In [ ]:
exoplanet_data, preferred_idx = query_nexsci_archive(target_name)
exoplanet_data

The preferred index variable (`preferred_idx`) is the value of the row from the table where the NexSci table column for `default_flag==1`. This are the preferred values for system, but others are available. 

In [ ]:
print(preferred_idx)

<a id='ltt1445ab_values'></a>
## Reviewing values returned from our API queries.

* Here we strip the units (degrees) from ra and dec. This is because the query of the MAST Archive (following cells) only accept python float types. If we pass the units still wrapped in the array (`exoplanet_data[preferred_idx]["ra"]`) the query will fail.
* For the values of `pl_tranmid` and `pl_orbper`, we keep the units (`d` which is short for `days`).

In [ ]:
ra = exoplanet_data[preferred_idx]["ra"].value[0]
dec = exoplanet_data[preferred_idx]["dec"].value[0]
T0 = exoplanet_data[preferred_idx]["pl_tranmid"][0]
P = exoplanet_data[preferred_idx]["pl_orbper"][0]
print(ra, dec, T0, P)

<a id='mastquery_ltt1445ab'></a>
## Query for JWST Observations of Target

In [ ]:
jwst_observations = check_jwst_observations(ra, dec)
jwst_observations

For the host `LTT 1445A` there are many observations, but for the exoplanet (`LTT 1445A b`), are there any events in the time period of the JWST observations listed above? We will use the columns `date_obs` and `duration` coupled with `pl_tranmid` and `pl_orbper` to calculate phase of the exoplanet which we will use to see if there is an event occuring during the time of the JWST observations in this table.

In [ ]:
observation_types = check_jwst_event_type(target_name, period=P, planet_ephemeris=T0, jwst_observations=jwst_observations)
observation_types

Let's add the observation types to our JWST observations Table

In [ ]:
jwst_observations.add_column(observation_types, name="obs_type")

<a id='ltt1445ab_results'></a>
## LTT 1445 A b Secondary Eclipse Results

For the Rocky Worlds DDT, we are interest in secondary eclipses, let's see if there is the possiblity for that type of event for our target in the observations returned from our MAST query.

In [ ]:
jwst_observations[jwst_observations["obs_type"]=="SECONDARY ECLIPSE"]

<a id='trappist1g'></a>
## Query NexSci Archive (TRAPPIST-1 g)

Now we are going to look at another Target Under Consideration, `TRAPPIST-1 g`. This target has issues with data in the NexSci Database, the preferred database entry is missing an important piece of data to calculate the phase.

**Note:** This notebook was written in January 2025, this issue could disappear if preferred data entry in the NexSci database is changed or if new measurements are made. This example is a work around to obtain the information we are interested in.

In [ ]:
failed_target_name = "TRAPPIST-1 g"

In [ ]:
failed_exoplanet_data, failed_preferred_idx = query_nexsci_archive(failed_target_name)
failed_exoplanet_data

<a id='trappist1g_values'></a>
## Reviewing values returned from our API queries.
Inspecting the metadata from the preferred row, the planetary ephemeris (`T0` or `pl_tranmid`) is NaN.

In [ ]:
failed_ra = failed_exoplanet_data[failed_preferred_idx]["ra"].value[0]
failed_dec = failed_exoplanet_data[failed_preferred_idx]["dec"].value[0]
failed_T0 = failed_exoplanet_data[failed_preferred_idx]["pl_tranmid"][0]
failed_P = failed_exoplanet_data[failed_preferred_idx]["pl_orbper"][0]
print(failed_ra, failed_dec, failed_T0, failed_P)

<a id='valid_values'></a>
## Selecting Valid Values for query_mast_jwst_archive

In this case, we can use the planet ephemeris from Gillon et al 2017.

In [ ]:
successful_T0 = failed_exoplanet_data[[0]]["pl_tranmid"][0]
successful_reference = failed_exoplanet_data[[0]]["pl_refname"][0]
print(successful_T0)
print(successful_reference)

In [ ]:
trappist1g_jwst_observations = check_jwst_observations(failed_ra, failed_dec)
trappist1g_jwst_observations

In [ ]:
trappist_1_g_event_types = check_jwst_event_type(failed_target_name, period=failed_P, planet_ephemeris=successful_T0, jwst_observations=trappist1g_jwst_observations)
trappist_1_g_event_types

In [ ]:
trappist1g_jwst_observations.add_column(trappist_1_g_event_types, name="obs_type")

<a id='trappist1g_results'></a>
## Trappist-1 g Secondary Eclipse Results

In [ ]:
trappist1g_jwst_observations[trappist1g_jwst_observations["obs_type"]=="SECONDARY ECLIPSE"]

<a id='all_events_for_tuc'></a>
## All Events for all Target Under Consideration

In [ ]:
planet_names = ["TOI-406.01", "TOI-198 b", "TOI-1452 b", "TOI-700 d", "LHS 1140 b", "TOI-776 b",
                "TOI-260 b", "LP 890-9 c", "K2-415 b", "TRAPPIST-1 g", "TOI-1467 b",
                "TOI-700 e", "TOI-1680 b", "TOI-244 b", "TOI-237 b", "LTT 1445 A b", "HD 260655 c",
                "K2-239 d", "TOI-5799 b", "TRAPPIST-1 f", "K2-129 b", "TOI-4559 b", "Kepler-445 b",
                "TOI-5388.01", "TOI-2266 b", "Gliese 12 b", "TOI-1634 b", "LHS 1140 c", "LP 890-9 b",
                "GJ 357 b", "TOI-1468 b", "TRAPPIST-1 h", "K2-239 c", "TOI-406 b", "LHS 1815 b",
                "K2-239 b", "TOI-2096 b", "L 98-59 d", "TOI-700 b", "L 98-59 c", "HD 260655 b",
                "TRAPPIST-1 e", "TOI-1693 b", "LHS 1478 b", "GJ 486 b", "TOI-270 b", "Kepler-446 b",
                "LTT 1445 A c", "TOI-771 b", "TRAPPIST-1 c", "LHS 1678 d", "GJ 1132 b", "GJ 3929 b",
                "TRAPPIST-1 b", "LP 791-18 d", "LHS 1678 c", "TOI-6086 b", "TOI-1450 A b", "GJ 3473 b",
                "LTT 3780 b", "TRAPPIST-1 d", "LP 791-18 b", "TOI-5720 b", "LHS 475 b", "TOI-1685 b",
                "GJ 806 b", "K2-91 b", "Wolf 327 b", "TOI-6008 b", "LHS 3844 b", "TOI-540 b", "SPECULOOS-3 b",
                "L 98-59 b", "Kepler-42 b", "GJ 1252 b", "TOI-4527.01", "LHS 1678 b", "GJ 238 b", "Kepler-42 c",
                "GJ 367 b",]

In [ ]:
all_observation_type_tables = []

for planet_name in planet_names:
    # Get exoplanet data from NexSci
    planet_data, _ = query_nexsci_archive(planet_name)
    # Get same data, but ensure planet ephemeris and period are NOT value NaN
    nan_free_planet_data = planet_data[~np.isnan(planet_data["pl_tranmid"]) & ~np.isnan(planet_data["pl_orbper"])]

    if len(nan_free_planet_data) == 0:
        print(f"No complete datasets for {planet_name}")
        continue

    # Check to see if preferred index is still in this updated table
    preferred_index = np.where(nan_free_planet_data["default_flag"]==1)[0]
    if preferred_index.size > 0:
        T0 = nan_free_planet_data[preferred_index]["pl_tranmid"][0]
        P = nan_free_planet_data[preferred_index]["pl_orbper"][0]
        ra = nan_free_planet_data[preferred_index]["ra"].value[0]
        dec = nan_free_planet_data[preferred_index]["dec"].value[0]
    else:
        # Just use first entry with complete data
        T0 = nan_free_planet_data[0]["pl_tranmid"]
        P = nan_free_planet_data[0]["pl_orbper"]
        ra = nan_free_planet_data[0]["ra"].value
        dec = nan_free_planet_data[0]["dec"].value

    jwst_observations = check_jwst_observations(ra, dec)
    if len(jwst_observations) == 0:
        print(f"No JWST observations for {planet_name}")
        continue
    else:
        print(f"Found {len(jwst_observations)} potential {planet_name} observations")

    observation_type_table = check_jwst_event_type(planet_name, period=P, planet_ephemeris=T0, jwst_observations=jwst_observations)
    jwst_observations.add_column(observation_type_table, name="obs_type")
    jwst_observations.add_column([planet_name] * len(observation_type_table), name="planet_name")
    all_observation_type_tables.append(jwst_observations)

combine_all_observation_tables = vstack(all_observation_type_tables)

In [ ]:
combine_all_observation_tables

In [ ]:
secondary_eclipses = np.where((combine_all_observation_tables["obs_type"] == "SECONDARY ECLIPSE")
                             & (combine_all_observation_tables["access"] == "PUBLIC"))
all_public_secondary_eclipses = combine_all_observation_tables[secondary_eclipses]["planet_name", "ArchiveFileID", "instrume", "exp_type", "opticalElements", "date_obs", "program", "access"]

In [ ]:
all_public_secondary_eclipses

In [ ]:
ascii.write(all_public_secondary_eclipses, 'all_public_secondary_eclipses.dat', overwrite=True)  